**Speech Denoising using RNN**

In [2]:
!pip install librosa # in colab, you'll need to install this
import librosa

import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

import matplotlib.pyplot as plt
import pylab as pl
import pandas as pd
import numpy as np
import pickle

from IPython.display import display, clear_output
from __future__ import print_function
from ipywidgets import interact, interactive, fixed
import ipywidgets as widgets
from math import ceil
from IPython.display import Audio
from scipy.io import wavfile
import math

%matplotlib inline

In [0]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [0]:
#Parameter values
numfiles = 1200
max_len = 250

In [8]:
from google.colab import files
uploaded = files.upload()

Saving trn0000.wav to trn0000.wav
Saving trn0001.wav to trn0001.wav
Saving trn0002.wav to trn0002.wav
Saving trn0003.wav to trn0003.wav
Saving trn0004.wav to trn0004.wav
Saving trn0005.wav to trn0005.wav
Saving trn0006.wav to trn0006.wav
Saving trn0007.wav to trn0007.wav
Saving trn0008.wav to trn0008.wav
Saving trn0009.wav to trn0009.wav
Saving trn0010.wav to trn0010.wav
Saving trn0011.wav to trn0011.wav
Saving trn0012.wav to trn0012.wav
Saving trn0013.wav to trn0013.wav
Saving trn0014.wav to trn0014.wav
Saving trn0015.wav to trn0015.wav
Saving trn0016.wav to trn0016.wav
Saving trn0017.wav to trn0017.wav
Saving trn0018.wav to trn0018.wav
Saving trn0019.wav to trn0019.wav
Saving trn0020.wav to trn0020.wav
Saving trn0021.wav to trn0021.wav
Saving trn0022.wav to trn0022.wav
Saving trn0023.wav to trn0023.wav
Saving trn0024.wav to trn0024.wav
Saving trn0025.wav to trn0025.wav
Saving trn0026.wav to trn0026.wav
Saving trn0027.wav to trn0027.wav
Saving trn0028.wav to trn0028.wav
Saving trn0029

In [0]:
#File loading and manipulation function

def speech_file_loading(tag):
    orig_signals = []
    stft_signals = []
    abs_signals = []
    lengths = []
    zeroes = ['000', '00', '0', '']
    
    for i in range(numfiles):
        if (i == 0):
            j = 0
        else:
            j = int(math.log10(i))
        
        s, sr = librosa.load(tag + zeroes[j] + str(i) + '.wav', sr = None)
        orig_signals.append(s)
        
        #Calculating STFT
        stft = librosa.stft(s, n_fft= 1024, hop_length= 512)
        stftlen = stft.shape[1]
        stft_signals.append(stft)
        
        #Calculating Magnitude values
        stftabs = np.abs(stft)
        stftabs = np.pad(stftabs, ((0,0),(0, max_len-stftlen)), 'constant')
        abs_signals.append(stftabs)
        
        lengths.append(stftlen)
        
    return orig_signals, stft_signals, abs_signals, lengths

In [0]:
#Loading speech signal files for training

trx_signal, X_inp, X_abs, X_len = speech_file_loading('trx')
trs_signal, S_inp, S_abs, S_len = speech_file_loading('trs')
trn_signal, N_inp, N_abs, N_len = speech_file_loading('trn')

In [11]:
len(X_inp), len(X_abs)
len(S_inp), len(S_abs)
len(N_inp), len(N_abs)

(1200, 1200)

(1200, 1200)

(1200, 1200)

In [0]:
#Function for Ideal Binary Masks (IBM)

def IBM(S, N):
    M = []    
    for i in range(len(S)):
        m = 1 * (S[i] > N[i])
        M.append(m)    
    return M

In [0]:
#Function to calculate SNR

def SNR_function(M, X, s, i):
    M = 1 * (M > 0.5)
    M = M.T
    S_pred = M * X
    s_pred = librosa.istft(S_pred, win_length = 1024, hop_length = 512)    
    nlen = min(len(s), len(s_pred))
    SNR = 10*math.log10((np.sum(s[:nlen]**2))/(np.sum((s[:nlen] - s_pred[:nlen])**2)))    
    return SNR

In [0]:
#Getting all Ideal Binary Masks
M = IBM(S_abs, N_abs)

In [15]:
#RNN for training

seq = tf.placeholder(tf.int32, None)
dropout_var = tf.placeholder(tf.float32, ())

X = tf.placeholder(tf.float32, [None, max_len, 513])
y = tf.placeholder(tf.float32, [None, max_len, 513])

#Defining the RNN
rnn_cell = tf.contrib.rnn.LayerNormBasicLSTMCell(1024, activation= tf.nn.relu, dropout_keep_prob = dropout_var)
rnn_output, rnn_state = tf.nn.dynamic_rnn(rnn_cell, X, dtype=tf.float32, sequence_length=seq)

dense_output = tf.layers.dense(rnn_output, 513, kernel_initializer= tf.contrib.layers.xavier_initializer())
output = tf.sigmoid(dense_output)

d = seq[0]

Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use keras.layers.dense instead.


In [16]:
#Cost and Optimization

cost = tf.reduce_mean(tf.losses.mean_squared_error(output[:, :d,:], y[:, :d, :]))
optimizer = tf.train.AdamOptimizer(learning_rate= 0.001).minimize(cost)

Instructions for updating:
Use tf.cast instead.


In [17]:
#Running the model for training
init = tf.global_variables_initializer()

max_epochs = 100
batch_size = 20
step = 5

sess = tf.Session()
sess.run(init)

for epoch in range(max_epochs):
    avg_cost = 0.
    random = np.arange(0, 1200, 20)
    np.random.shuffle(random)

    for i in range(len(random)):
        start = int(random[i])
        end = int(start + batch_size)
        b_x, b_y = np.array(X_abs[start:end]).swapaxes(1,2), np.array(M[start:end]).swapaxes(1,2)
        seqlen = np.array(X_len[start:end])
        data = {X: b_x, y: b_y, seq : seqlen, dropout_var : 0.7}
        sess.run(optimizer, feed_dict=data)
        avg_cost += sess.run(cost, feed_dict=data)
     
    avg_cost = avg_cost / len(random)
        
    if (epoch+1) % step == 0:
        print ("Epoch: %03d/%03d cost: %.9f" % (epoch, max_epochs, avg_cost))
        
print ("=========================Model Optimization Complete============================")

Epoch: 004/100 cost: 0.168806419
Epoch: 009/100 cost: 0.146125946
Epoch: 014/100 cost: 0.135733013
Epoch: 019/100 cost: 0.127542254
Epoch: 024/100 cost: 0.120368258
Epoch: 029/100 cost: 0.114627862
Epoch: 034/100 cost: 0.109056722
Epoch: 039/100 cost: 0.106595182
Epoch: 044/100 cost: 0.101820103
Epoch: 049/100 cost: 0.098227725
Epoch: 054/100 cost: 0.095650748
Epoch: 059/100 cost: 0.092977379
Epoch: 064/100 cost: 0.090706278
Epoch: 069/100 cost: 0.090833027
Epoch: 074/100 cost: 0.087685343
Epoch: 079/100 cost: 0.086193129
Epoch: 084/100 cost: 0.085210950
Epoch: 089/100 cost: 0.084277204
Epoch: 094/100 cost: 0.082380696
Epoch: 099/100 cost: 0.081351506
=========================Model Optimization Complete============================


In [18]:
#Getting SNR values and mean SNR for training dataset

SNR_train = np.zeros(1200)

for i in range(len(X_abs)):
    b_x = np.zeros((1, X_abs[i].shape[1], X_abs[i].shape[0]))
    b_y = np.zeros((1, X_abs[i].shape[1], X_abs[i].shape[0]))
    
    b_x[0,:,:] = X_abs[i].T
    b_y[0,:,:] = S_abs[i].T
    
    M_pred, loss = sess.run([output, cost], feed_dict = {X:b_x, y:b_y, seq : np.array([X_len[i]]), dropout_var:1})
    
    SNR_train[i] = SNR_function(M_pred[0,:X_len[i],:], X_inp[i], trs_signal[i], i)
    
print(np.mean(SNR_train))

12.622878587218812


**Speech denoising on samples (800) in validation set using trained RNN**

In [22]:
from google.colab import files
uploaded = files.upload()

Saving vx0000.wav to vx0000.wav
Saving vx0001.wav to vx0001.wav
Saving vx0002.wav to vx0002.wav
Saving vx0003.wav to vx0003.wav
Saving vx0004.wav to vx0004.wav
Saving vx0005.wav to vx0005.wav
Saving vx0006.wav to vx0006.wav
Saving vx0007.wav to vx0007.wav
Saving vx0008.wav to vx0008.wav
Saving vx0009.wav to vx0009.wav
Saving vx0010.wav to vx0010.wav
Saving vx0011.wav to vx0011.wav
Saving vx0012.wav to vx0012.wav
Saving vx0013.wav to vx0013.wav
Saving vx0014.wav to vx0014.wav
Saving vx0015.wav to vx0015.wav
Saving vx0016.wav to vx0016.wav
Saving vx0017.wav to vx0017.wav
Saving vx0018.wav to vx0018.wav
Saving vx0019.wav to vx0019.wav
Saving vx0020.wav to vx0020.wav
Saving vx0021.wav to vx0021.wav
Saving vx0022.wav to vx0022.wav
Saving vx0023.wav to vx0023.wav
Saving vx0024.wav to vx0024.wav
Saving vx0025.wav to vx0025.wav
Saving vx0026.wav to vx0026.wav
Saving vx0027.wav to vx0027.wav
Saving vx0028.wav to vx0028.wav
Saving vx0029.wav to vx0029.wav
Saving vx0030.wav to vx0030.wav
Saving v

In [0]:
#Loading speech signal files for validation

numfiles = 800

valx_signal, ValX_inp, ValX_abs, ValX_len = speech_file_loading('vx')
vals_signal, ValS_inp, ValS_abs, ValS_len = speech_file_loading('vs')
valn_signal, ValN_inp, ValN_abs, ValN_len = speech_file_loading('vn')

In [24]:
len(ValX_inp), len(ValX_abs)
len(ValS_inp), len(ValS_abs)
len(ValN_inp), len(ValN_abs)

(800, 800)

(800, 800)

(800, 800)

In [0]:
#Ideal Binary Masks for validation sets
Val_M = IBM(ValS_abs, ValN_abs)

In [26]:
#Getting SNR values and mean SNR for validation dataset

SNR_valid = np.zeros(800)

for i in range(len(ValX_abs)):
    b_x = np.zeros((1, ValX_abs[i].shape[1], ValX_abs[i].shape[0]))
    b_y = np.zeros((1, ValX_abs[i].shape[1], ValX_abs[i].shape[0]))
    
    b_x[0,:,:] = ValX_abs[i].T
    b_y[0,:,:] = ValS_abs[i].T
    
    M_pred, loss = sess.run([output, cost], feed_dict = {X:b_x, y:b_y, seq : np.array([ValX_len[i]]), dropout_var:1})
    
    SNR_valid[i] = SNR_function(M_pred[0,:ValX_len[i],:], ValX_inp[i], vals_signal[i], i)
    
print(np.mean(SNR_valid))

11.644615580306002


**Speech denoising on test data using trained RNN**

In [27]:
from google.colab import files
uploaded = files.upload()

Saving tex0000.wav to tex0000.wav
Saving tex0001.wav to tex0001.wav
Saving tex0002.wav to tex0002.wav
Saving tex0003.wav to tex0003.wav
Saving tex0004.wav to tex0004.wav
Saving tex0005.wav to tex0005.wav
Saving tex0006.wav to tex0006.wav
Saving tex0007.wav to tex0007.wav
Saving tex0008.wav to tex0008.wav
Saving tex0009.wav to tex0009.wav
Saving tex0010.wav to tex0010.wav
Saving tex0011.wav to tex0011.wav
Saving tex0012.wav to tex0012.wav
Saving tex0013.wav to tex0013.wav
Saving tex0014.wav to tex0014.wav
Saving tex0015.wav to tex0015.wav
Saving tex0016.wav to tex0016.wav
Saving tex0017.wav to tex0017.wav
Saving tex0018.wav to tex0018.wav
Saving tex0019.wav to tex0019.wav
Saving tex0020.wav to tex0020.wav
Saving tex0021.wav to tex0021.wav
Saving tex0022.wav to tex0022.wav
Saving tex0023.wav to tex0023.wav
Saving tex0024.wav to tex0024.wav
Saving tex0025.wav to tex0025.wav
Saving tex0026.wav to tex0026.wav
Saving tex0027.wav to tex0027.wav
Saving tex0028.wav to tex0028.wav
Saving tex0029

In [0]:
numfiles = 400
testx_signal, TestX_inp, TestX_abs, TestX_len = speech_file_loading('tex')

In [0]:
#Getting de-noised test signals and saving them
sr= 16000

for i in range(len(TestX_abs)):
    b_x = np.zeros((1, TestX_abs[i].shape[1], TestX_abs[i].shape[0]))
    
    b_x[0,:,:] = TestX_abs[i].T
    
    M_test_pred = sess.run(output, feed_dict = {X:b_x, seq : np.array([TestX_len[i]]), dropout_var:1})
    M_test = M_test_pred[0,:TestX_len[i],:]
    
    M_test = 1 * (M_test > 0.5)
    M_test = M_test.T
    S_test = M_test * TestX_inp[i]
    s_test_pred = librosa.istft(S_test, win_length = 1024, hop_length = 512)
    
    librosa.output.write_wav('test_' + str(i) + '.wav', s_test_pred, sr)
    
    files.download('test_' + str(i) + '.wav')
    